Aluno: Pedro Rodrigues Corrêa - 243236

## Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Portanto, você deve implementar o modelo de linguagem inspirado no artigo do Bengio, para prever a próxima palavra usando rede com embeddings e duas camadas.
Sugestão de alguns parâmetros:
* context_size = 9
* max_vocab_size = 3000
* embedding_dim = 64
* usar pontuação no vocabulário
* descartar qualquer contexto ou target que não esteja no vocabulário
* É esperado conseguir uma perplexidade da ordem de 50.
* Procurem fazer asserts para garantir que partes do seu programa estão testadas

Este enunciado não é fixo, podem mudar qualquer um dos parâmetros acima, mas procurem conseguir a perplexidade esperada ou menor.

Gerem alguns frases usando um contexto inicial e depois deslocando o contexto e prevendo a próxima palavra gerando frases compridas para ver se está gerando texto plausível.

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída).
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

Procure por `TODO` para entender onde você precisa inserir o seu código.

## Faz download e carrega o dataset

In [ ]:
!wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
!wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import sklearn


In [ ]:
text = open("pg67724.txt","r").read()
text += open("pg67725.txt","r").read()

paragraphs = text.split("\n\n")
len(paragraphs)

4969

In [ ]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

for _ in cleaned_paragraphs[100:110]:
  print(_)

len(cleaned_paragraphs)

 III
A BANDEIRA
 Era meio dia.
Um troço de cavalleiros, que constaria quando muito de quinze pessoas, costeava a margem direita do Parahyba.
Estavão todos armados da cabeça até aos pés; além da grande espada de guerra que batia as ancas do animal, cada um delles trazia á cinta dous pistoletes, um punhal na ilharga do calção, e o arcabuz passado a tiracollo pelo hombro esquerdo.
Pouco adiante, dous homens a pé tocavão alguns animaes carregados de caixas e outros volumes cobertos com uma sarapilheira alcatroada, que os abrigava da chuva.
Quando os cavalleiros, que seguião a trote largo, vencião a pequena distancia que os separava da tropa, os dous caminheiros, para não atrazarem a marcha, montavão na garupa dos animaes e ganhavão de novo a dianteira.
Naquelle tempo dava-se o nome de _bandeiras_ a essas caravanas de aventureiros que se entranhavão pelos sertões do Brasil, á busca de ouro, de brilhantes e esmeraldas, ou á descoberta de rios e terras ainda desconhecidos. A que nesse momento

4892

## Análise do dataset

In [ ]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+|[.,!?-]', text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)
print(word_counts)

Counter({'.': 8870, ',': 7693, '-': 6223, 'a': 4595, 'que': 4340, 'o': 4079, 'de': 3960, 'e': 3658, 'se': 2402, 'um': 1711, 'do': 1442, 'não': 1280, 'uma': 1250, 'da': 1133, 'os': 1123, 'com': 1015, 'sua': 925, 'para': 857, 'seu': 777, '!': 773, 'pery': 732, 'as': 726, 'em': 724, 'no': 664, '?': 628, 'por': 622, 'ao': 594, 'como': 594, 'lhe': 558, 'd': 493, 'á': 490, 'tinha': 478, 'era': 469, 'cecilia': 457, 'na': 455, 'é': 441, 'sobre': 416, 'mas': 410, 'elle': 407, 'the': 376, 'dos': 373, 'indio': 340, 'me': 325, 'seus': 324, 'mais': 318, 'antonio': 303, 'quando': 288, 'alvaro': 278, 'disse': 259, 'das': 258, 'vos': 254, 'of': 252, 'ella': 233, 'olhos': 227, 'te': 227, 'senhora': 227, 'menina': 215, 'pela': 213, 'tu': 204, 'depois': 200, 'nos': 200, 'isabel': 197, 'havia': 195, 'gutenberg': 194, 'fidalgo': 194, 'casa': 192, 'estava': 187, 'ainda': 186, 'tempo': 182, 'já': 181, 'mariz': 180, 'project': 176, 'aventureiros': 175, 'momento': 174, 'loredano': 174, 'só': 173, 'mesmo': 173,

## Criando um vocabulário

In [ ]:
vocab_size = 3000

UNK = '<UNK>'
most_frequent_words = [UNK] + [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words)}
vocab_size = len(vocab)

In [ ]:
len(vocab)

3001

In [ ]:
import re

# Function to encode a sentence into a list of indices based on a vocabulary
def encode_sentence(sentence, vocab):
    # Tokenize the sentence into words and punctuation marks
    tokens = re.findall(r'\w+|[.,!?-]', sentence.lower())
    # Encode each token using the vocabulary, replacing unknown words with 0
    encoded_sentence = [vocab.get(word, 0) for word in tokens]
    return encoded_sentence

# Function to decode a list of indices into a sentence using a vocabulary
def decode_sentence(encoded_sentence, vocab):
    words = []
    # Iterate through each index in the encoded sentence
    for index in encoded_sentence:
        # Find the corresponding word in the vocabulary for the index
        # If the index is not found in the vocabulary, replace it with "<UNK>"
        word = next((word for word, code in vocab.items() if code == index), "<UNK>")
        words.append(word)
    return words


## Dataset

In [ ]:
def gera_input_target(text, context_size):
    # Initialize lists to store contexts and targets.
    contexts = []
    targets = []

    # Iterate over the text to generate contexts and corresponding targets.
    for i in range(len(text) - context_size):
        # Extract the context of size 'context_size' starting from index 'i'.
        context = text[i: i + context_size]
        # Retrieve the target element immediately following the context.
        target = text[i + context_size]
        # Append the context and target to their respective lists.
        contexts.append(context)
        targets.append(target)

    # Return the lists of contexts and targets.
    return contexts, targets


In [ ]:
class MyDataset(Dataset):
    def __init__(self, text, vocab, context_size=9, vocab_size=3000):
        # Initialize the dataset with the provided text, vocabulary, and context size.
        # Encode each sentence in the text using the provided vocabulary.
        self.vocab = vocab
        self.context_size = context_size
        self.data = [encode_sentence(sentence, self.vocab) for sentence in text]

        # Initialize lists to store contexts and targets.
        contexts_list = []
        targets_list = []

        # Iterate over the encoded data to generate inputs and targets.
        for coded in self.data:
            # Skip sentences shorter than the context size.
            if len(coded) < self.context_size:
                continue

            # Check if any token in the encoded sentence is unknown (UNK).
            if any(token == self.vocab[UNK] for token in coded):
                continue  # Skip this example if it contains unknown tokens.

            # Generate inputs and targets using the current sentence's encoded representation.
            inputs, targets = gera_input_target(coded, context_size)
            contexts_list.extend(inputs)
            targets_list.extend(targets)

        # Convert the lists of contexts and targets into tensors.
        self.contexts_tensor = torch.tensor(contexts_list)
        self.targets_tensor = torch.tensor(targets_list)

    def __len__(self):
        # Return the total number of samples in the dataset.
        return len(self.targets_tensor)

    def __getitem__(self, idx):
        # Retrieve and return the context and target tensors for the given index.
        return self.contexts_tensor[idx], self.targets_tensor[idx]


In [ ]:
from sklearn.model_selection import train_test_split

train_text, test_text = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

In [ ]:
# Gera os dataset de treino e validação
train_dataset = MyDataset(train_text, vocab)
test_dataset = MyDataset(test_text, vocab)

In [ ]:
batch_size = 32
# Criando instâncias de DataLoader para treinamento e validação
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False)
sample = next(iter(train_loader))

## Model

In [ ]:
import torch
import torch.nn as nn

# Define the LanguageModel class, inheriting from nn.Module
class LanguageModel(nn.Module):

    # Constructor method, initializes the model parameters
    def __init__(
        self,
        vocab_size = len(vocab),  # Vocabulary size (assuming `vocab` is defined elsewhere)
        context_size = 9,          # Context window size
        embedding_dim = 64,        # Dimensionality of word embeddings
        hidden = 500               # Dimensionality of hidden layer in MLP
    ):
        super().__init__()  # Call the constructor of the parent class

        self.context_size = context_size  # Store the context window size

        # Define the embedding layer to map word indices to embedding vectors
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Define the MLP (Multi-Layer Perceptron) neural network
        self.mlp = nn.Sequential(
            # Linear layer to transform the concatenated embedding vectors into hidden layer
            nn.Linear(context_size * embedding_dim, hidden),
            nn.Tanh(),  # Hyperbolic tangent activation function
            # Linear layer to transform hidden layer output into output layer
            nn.Linear(hidden, vocab_size)  # Output size is vocab_size + 1
        )

    # Forward method defines how input data is processed through the model
    def forward(self, x):
        o = self.embedding(x)  # Lookup embeddings for input words
        o = o.flatten(start_dim=1)  # Flatten the embeddings into a single vector
        return self.mlp(o)  # Pass the flattened vector through the MLP

# Create an instance of the LanguageModel class with specified parameters
model = LanguageModel(vocab_size)

In [ ]:
print(model)

LanguageModel(
  (embedding): Embedding(3001, 64)
  (mlp): Sequential(
    (0): Linear(in_features=576, out_features=500, bias=True)
    (1): Tanh()
    (2): Linear(in_features=500, out_features=3001, bias=True)
  )
)


In [ ]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

In [ ]:
print(model(input).shape)

torch.Size([32, 3001])


In [ ]:
input.shape

torch.Size([32, 9])

In [ ]:
output = model(input)

In [ ]:
output.argmax(dim=1)

tensor([2885,  734,  934,   84,  678, 1287,  561, 1246, 1554,  276, 2122,  208,
        2144, 2352,  212, 1826, 2745, 1942, 1799,  775,  207,  345,  886, 1484,
         727,  623, 2325,  896,  951, 1033, 2560,  714])

In [ ]:
target

tensor([  19,    3,    4,    2,  641, 1067,  912,  167,  552,   24,  256,  253,
          15,    7,   33,  110,    1,    1,    7,   15,  413,  673,    2,   25,
          76,    2,  875,   52,   19,  183,  260,    5])

## Training

In [ ]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.to(device)

# Calculate loss before training
model.eval()  # Set the model to evaluation mode
initial_loss = 0
with torch.no_grad():
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        logits = model(inputs)
        initial_loss += criterion(logits, targets)

    avg_loss = initial_loss / len(train_loader)

initial_PPL = torch.exp(avg_loss)
print(f'Initial Loss: {avg_loss:.4f}, n\
        Initial Perplexity: {initial_PPL}')

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        logits = model(inputs)
        loss_train = criterion(logits, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        ppl_train = torch.exp(loss_train)

    model.eval()

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            logits = model(inputs)

    loss_test = criterion(logits, targets)
    ppl_test = torch.exp(loss_test)

    print(f'Epoch [{epoch+1}/{num_epochs}], \
            Loss Treinamento: {loss_train.item():.4f}, \
            PPL Treinamento: {ppl_train.item():.4f}, \
            Loss Teste: {loss_test.item():.4f}, \
            PPL Teste: {ppl_test.item():.4f}')

Initial Loss: 8.0324, n        Initial Perplexity: 3079.132080078125
Epoch [1/5],             Loss Treinamento: 4.6124,             PPL Treinamento: 100.7245,             Loss Teste: 6.1223,             PPL Teste: 455.9276
Epoch [2/5],             Loss Treinamento: 5.4192,             PPL Treinamento: 225.6927,             Loss Teste: 4.9730,             PPL Teste: 144.4539
Epoch [3/5],             Loss Treinamento: 6.9961,             PPL Treinamento: 1092.3594,             Loss Teste: 4.7264,             PPL Teste: 112.8887
Epoch [4/5],             Loss Treinamento: 1.7568,             PPL Treinamento: 5.7937,             Loss Teste: 4.5402,             PPL Teste: 93.7132
Epoch [5/5],             Loss Treinamento: 1.0102,             PPL Treinamento: 2.7461,             Loss Teste: 4.6927,             PPL Teste: 109.1510


## Exemplo de uso

In [ ]:
import random

def generate_text(model: LanguageModel, length: int, vocab: vocab):

    # Ensure that the length is at least equal to the context size
    assert length > model.context_size

    # Initialize the sentence with random words from the vocabulary
    sentence = random.sample(range(0, vocab_size), model.context_size)

    with torch.no_grad():
        while len(sentence) < length:
            x = torch.unsqueeze(torch.asarray(sentence[-model.context_size:]), dim=0).to(device)
            y = model(x).squeeze()
            y = nn.Softmax(dim=0)(y)

            # Choose the next word according to probabilities
            y = random.choices(range(0, vocab_size), y)
            sentence.append(y[0])

    return ' '.join(decode_sentence(sentence, vocab))


context_size = 5
max_length= 10
generate_text(model, max_length, vocab)

'santa joão traz graciosas sebastião impressão procurava comsigo hesitou comprehendêra'

## Referências

Leandro Carísio Fernandes

Ramon Simões Abilio

Fernando Gubitoso Marques